In [9]:
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
import cv2 as cv
from matplotlib import pyplot as plt
import matplotlib as mpl
import plotly.express as px

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
from functools import partial

In [2]:
classes = pd.read_csv('classes.csv', sep=',')
all_names = {}
classes["class"] = np.zeros(classes.shape[0])
for ind, col in enumerate(classes.columns[1:8]):
    print(col, ind)
    classes["class"][classes[col]==1] = ind
    all_names[col] = list(classes["image"][classes[col]==1])

MEL 0
NV 1
BCC 2
AKIEC 3
BKL 4
DF 5
VASC 6


/tmp/ipykernel_220760/2670562764.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  classes["class"][classes[col]==1] = ind
/tmp/ipykernel_220760/2670562764.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  classes["class"][classes[col]==1] = ind
/tmp/ipykernel_220760/2670562764.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  classes["class"][classes[col]==1] = ind
/tmp/ipykernel_220760/2670562764.py:6: SettingWithCopyWarning: 


In [3]:
class_names = classes.columns[1:8]
for col in class_names:
    print(col, classes[col].sum())

MEL 1113.0
NV 6705.0
BCC 514.0
AKIEC 327.0
BKL 1099.0
DF 115.0
VASC 142.0


In [7]:
def get_img(name) :
    return cv.imread("pictures/" + name + ".jpg").astype(int)

def calc_square_means(img, val) : 
    bot, top = 225-3*val, 225+3*val
    left, right = 300-4*val, 300+4*val
    sq = img[bot:top, left:right]
    return np.r_[sq.mean(axis=(0,1)), sq.std(axis=(0,1))]

def add_new_columns():
    means = []
    stds = []
    sq_res = []
    for name in tqdm(classes["image"]):
        img = get_img(name)
        means.append(img.mean(axis = (0,1)))
        stds.append(img.std(axis=(0,1)))
        sq_res.append(calc_square_means(img, 40))
    #res = pd.DataFrame(res, columns = ["mean0", "mean1", "mean2"])
    res = np.array(means)
    classes["mean0"] = res[:,0]
    classes["mean1"] = res[:,1]
    classes["mean2"] = res[:,2]
    res = np.array(stds)
    classes["std0"] = res[:,0]
    classes["std1"] = res[:,1]
    classes["std2"] = res[:,2]
    res = np.array(sq_res)
    classes["sq_mean0"] = res[:,0]
    classes["sq_mean1"] = res[:,1]
    classes["sq_mean2"] = res[:,2]
    classes["sq_std0"] = res[:,3]
    classes["sq_std1"] = res[:,4]
    classes["sq_std2"] = res[:,5]

In [8]:
add_new_columns() #takes a few minutes

100%|█████████████████████████████████████| 10015/10015 [04:56<00:00, 33.83it/s]


In [10]:
%%time
my_columns5 = ["mean0", "std0", "sq_mean0", "sq_std0", "mean1", "std1", "sq_mean1", "sq_std1", "mean2", "std2"]

X = classes[my_columns5]
y = classes["MEL"]
X_train, X_test, y_train, y_test = train_test_split(X,y)
model = LogisticRegression(class_weight="balanced", max_iter = 2000)
model.fit(X_train, y_train)

CPU times: user 534 ms, sys: 1.25 s, total: 1.78 s
Wall time: 191 ms


LogisticRegression(class_weight='balanced', max_iter=2000)

In [11]:
y_pred = model.predict(X_test)
y0 = 1-y_pred[y_test==0]
accuracy0 = y0.sum() / y0.shape[0]
y1 = y_pred[y_test==1]
accuracy1 = y1.sum()/y1.shape[0]
print(f"Accuracy: {accuracy0:.3}, {accuracy1:.3}")

Accuracy: 0.716, 0.755
